# Engineering Calculations Essentials Notebook

**A Quick Reference for Civil/Structural Engineers Learning Python**

This notebook demonstrates core Python skills through practical engineering workflows. Each section builds on fundamental concepts:
- Defining and calling functions
- Working with data structures (lists, dictionaries, DataFrames)
- Performing calculations and visualizations
- Converting units and organizing engineering data

**How Jupyter Notebooks Work:**
- Notebooks are composed of **cells** that run sequentially from top to bottom
- **Code cells** execute Python commands when you run them (Shift+Enter)
- **Markdown cells** (like this one) provide formatted text and documentation
- Variables and functions defined in earlier cells remain available in later cells
- You can re-run cells in any order, but for consistency, run them top-to-bottom initially

Let's get started!

## 0. Library Imports & Setup

**Why import libraries at the start?**

In Python, we import external libraries to access pre-built tools and functions. By convention, all imports go at the top of a script or notebook so:
1. You can see all dependencies at a glance
2. Functions become available for all cells that follow
3. It's easier to manage and troubleshoot

**The libraries we're using:**
- **`matplotlib.pyplot`**: For creating plots and charts (imported as `plt` for convenience)
- **`numpy`**: For numerical arrays and mathematical operations (imported as `np`)
- **`pandas`**: For working with tables and structured data (imported as `pd`)
- **`math`**: Python's built-in math module (used later for π, sqrt, etc.)

The `%matplotlib inline` command tells Jupyter to display plots directly in the notebook.

In [ ]:
# Import libraries we'll use throughout this notebook
# These must be run BEFORE any cells that use them
import matplotlib.pyplot as plt  # plotting and visualization
import numpy as np               # numerical computing and arrays
import pandas as pd              # data tables and analysis

# Special Jupyter command to display plots inline
%matplotlib inline

## 1. Defining Functions: Structural Engineering Formulas

**Key Python Concepts Demonstrated:**
- **Function definition**: Using `def function_name(parameters):` to create reusable calculations
- **Docstrings**: Triple-quoted strings (`"""..."""`) that document what each function does
- **Parameters and return values**: Functions take inputs and give back results
- **Type hints in comments**: Units are noted in docstrings for clarity (e.g., `[m^2]`, `[Pa]`)

**Why use functions?**
- Avoid repeating the same calculation code
- Make your work easier to test and debug
- Create a library of standard formulas you can reuse across projects

The formulas below should look familiar, they're standard structural mechanics equations. Pay attention to:
1. How the formula is translated into Python syntax
2. How parameters are named clearly (e.g., `b`, `h`, `M`, `c`, `I`)
3. How each function returns a single value or tuple of values
4. How we separate *defining* functions from *calling* them (functions are defined first, then used later)

In [ ]:
# Import math module for mathematical constants and functions (pi, sqrt, etc.)
import math

# ============================================================================
# PART A: FUNCTION DEFINITIONS (Reusable formulas)
# ============================================================================
# Note: Defining a function does NOT execute it, it just makes it available.
# We'll call these functions later in PART B.

# 1) Rectangular area
def area_rect(b, h):  
    """Area of a rectangle, A = b*h  [m^2]."""
    return b * h

# 2) Second moment of area (rectangle about its centroidal axis)
def inertia_rect(b, h):
    """I (about centroidal axis parallel to b), I = b*h^3/12  [m^4]."""
    return b * h**3 / 12.0

# 3) Section modulus for rectangular section
def section_modulus_rect(b, h):
    """S = I / c = (b*h^3/12) / (h/2) = b*h^2/6  [m^3]."""
    return b * h**2 / 6.0

# 4) Bending stress at extreme fiber
def bending_stress(M, c, I):
    """σ = M*c / I  [Pa = N/m^2]. M in N·m, c in m, I in m^4."""
    return M * c / I

# 5) Maximum shear stress for a solid rectangle
def max_shear_stress_rect(V, b, h):
    """τ_max = 1.5 * V / A for a solid rectangular section  [Pa]."""
    A = b * h
    return 1.5 * V / A

# 6) Shear flow at a horizontal interface
def shear_flow(V, Q, I):
    """
    q = V*Q / I  [N/m]
    V: shear force (N)
    Q: first moment of area about NA for the portion above/below the cut (m^3)
    I: second moment of area (m^4)
    """
    return V * Q / I

# 7) Euler elastic buckling load (pinned-pinned column)
def euler_Pcr(E, I, L, K=1.0):
    """
    Pcr = π^2 * E * I / (K*L)^2  [N]
    K: effective length factor (default 1.0 for pinned-pinned)
    """
    return (math.pi**2) * E * I / (K * L)**2

# 8) Simply supported beam with UDL: support reactions
def ss_udl_reaction(w, L):
    """Each support reaction R = w*L/2  [N]."""
    return w * L / 2.0

# 9) Simply supported beam with UDL: maximum moment
def ss_udl_Mmax(w, L):
    """M_max = w*L^2/8  [N·m]."""
    return w * L**2 / 8.0

# 10) Simply supported beam with UDL: maximum deflection
def ss_udl_delta_max(w, L, E, I):
    """δ_max = 5*w*L^4 / (384*E*I)  [m]."""
    return 5.0 * w * L**4 / (384.0 * E * I)

# 11) Simply supported beam with midspan point load: max moment
def ss_point_Mmax(P, L):
    """M_max = P*L/4  [N·m]."""
    return P * L / 4.0

# 12) Simply supported beam with midspan point load: max deflection
def ss_point_delta_max(P, L, E, I):
    """δ_max = P*L^3 / (48*E*I)  [m]."""
    return P * L**3 / (48.0 * E * I)

# 13) Concrete elastic modulus (ACI approximation, SI units)
def Ec_from_fc(fc_MPa):
    """
    Ec ≈ 4700 * sqrt(fc')  [MPa] -> returns Pa
    Common approximation per ACI 318 (use your office standard as needed)
    """
    Ec_MPa = 4700.0 * math.sqrt(fc_MPa)
    return Ec_MPa * 1e6  # Convert MPa to Pa

# 14) Modulus of rupture (ACI 318, SI units)
def fr_from_fc(fc_MPa):
    """fr = 0.62 * sqrt(fc' [MPa])  [MPa] -> returns Pa."""
    fr_MPa = 0.62 * math.sqrt(fc_MPa)
    return fr_MPa * 1e6  # Convert MPa to Pa

# 15) Singly reinforced RC rectangular section: nominal moment capacity
def rc_flexure_Mn_phiMn(fc_MPa, fy_MPa, b, d, As, phi=0.90):
    """
    Nominal flexural strength (singly reinforced, rectangular):
      a = As*fy / (0.85*fc'*b)
      Mn = As*fy * (d - a/2)
    
    Returns: (Mn [N·m], phi*Mn [N·m])
    
    Inputs:
      fc_MPa, fy_MPa: concrete and steel strength in MPa
      b, d: width and effective depth in m
      As: area of tension steel in m^2
      phi: strength reduction factor (default 0.90 for tension-controlled)
    
    Note: Always verify strain limits for correct φ factor in real design.
    """
    fc = fc_MPa * 1e6  # Convert to Pa
    fy = fy_MPa * 1e6  # Convert to Pa
    a = As * fy / (0.85 * fc * b)  # Depth of equivalent stress block
    Mn = As * fy * (d - a / 2.0)    # Nominal moment capacity
    return Mn, phi * Mn


# ============================================================================
# PART B: CALLING FUNCTIONS (Running example calculations)
# ============================================================================
# Now we USE the functions defined above by calling them with actual values.
# This demonstrates how to pass arguments and capture returned results.

print("=" * 70)
print("EXAMPLE CALCULATIONS: 15 Structural Engineering Formulas")
print("=" * 70)

# Define example geometry and material properties
b, h, L = 0.2, 0.4, 6.0        # beam width, height, span (m)
c = h / 2.0                    # extreme fiber distance from NA (m)
I = inertia_rect(b, h)         # call function to compute I (m^4)
S = section_modulus_rect(b, h) # call function to compute S (m^3)
w = 12e3                       # uniform distributed load (N/m) = 12 kN/m
P = 50e3                       # point load at midspan (N) = 50 kN
E = 30e9                       # elastic modulus (Pa) = 30 GPa
V = 80e3                       # shear force (N) = 80 kN
fc, fy = 30.0, 500.0           # concrete and steel strength (MPa)
As = 800e-6                    # area of tension steel (m^2) = 800 mm^2

# Now call each function and print results
# Note the syntax: function_name(argument1, argument2, ...)

print("\n--- Geometry Properties ---")
print(f"1) Rectangular Area = {area_rect(b, h):.6f} m^2")
print(f"2) Moment of Inertia (I) = {I:.6e} m^4")
print(f"3) Section Modulus (S) = {S:.6e} m^3")

print("\n--- Stress Calculations ---")
M_udl = ss_udl_Mmax(w, L)  # First calculate moment, then use it for stress
print(f"4) Bending Stress (from UDL Mmax) = {bending_stress(M_udl, c, I):.2f} Pa")
print(f"5) Max Shear Stress (rectangular) = {max_shear_stress_rect(V, b, h):.2f} Pa")

# Shear flow example: first moment Q for top half of rectangle about NA
# For rectangle: top half area = b*h/2, centroid at h/4 from NA
Q_top_half = (b * h / 2.0) * (h / 4.0)
print(f"6) Shear Flow (at mid-height) = {shear_flow(V, Q_top_half, I):.2f} N/m")

print("\n--- Column Buckling ---")
print(f"7) Euler Buckling Load (Pcr, K=1) = {euler_Pcr(E, I, L):.2f} N")

print("\n--- Simply Supported Beam: Uniform Load ---")
print(f"8) Support Reaction = {ss_udl_reaction(w, L):.2f} N")
print(f"9) Max Moment = {M_udl:.2f} N·m")
print(f"10) Max Deflection = {ss_udl_delta_max(w, L, E, I):.6f} m")

print("\n--- Simply Supported Beam: Midspan Point Load ---")
print(f"11) Max Moment = {ss_point_Mmax(P, L):.2f} N·m")
print(f"12) Max Deflection = {ss_point_delta_max(P, L, E, I):.6f} m")

print("\n--- Concrete Material Properties ---")
print(f"13) Elastic Modulus (Ec from f'c) = {Ec_from_fc(fc):.2e} Pa")
print(f"14) Modulus of Rupture (fr from f'c) = {fr_from_fc(fc):.2e} Pa")

print("\n--- Reinforced Concrete Flexure ---")
Mn, phiMn = rc_flexure_Mn_phiMn(fc, fy, b, d=0.36, As=As)
print(f"15) Nominal Moment (Mn) = {Mn:.2f} N·m")
print(f"    Design Moment (φMn) = {phiMn:.2f} N·m")

print("=" * 70)

## 2. Unit Conversion with Dictionaries

**Key Python Concepts:**
- **Dictionaries**: Key-value pairs using `{key: value}` syntax
- **Tuples as keys**: Using `(from_unit, to_unit)` as dictionary keys
- **List comprehensions**: Compact syntax for transforming lists in one line
- **F-strings**: Modern string formatting with `f"{variable:.2f}"` for controlling decimal places

**Practical Application:**
Unit conversion is essential in engineering. This example shows how to create a simple conversion tool using a dictionary to store conversion factors. The `convert()` function looks up the appropriate factor and applies it.

**Note on Python syntax:**
- `1_000` is the same as `1000`, underscores make large numbers more readable
- `1 / 1_000` creates a float division (0.001) automatically in Python 3
- List comprehensions `[convert(x, ...) for x in list]` are a Pythonic way to apply a function to every item

In [ ]:
# Define a dictionary of unit conversion factors
# Key = (from_unit, to_unit), Value = multiplication factor
UNIT_FACTORS = {
    ("kN", "N"): 1_000,           # kilonewtons to newtons
    ("N", "kN"): 1 / 1_000,       # newtons to kilonewtons
    ("m", "mm"): 1_000,           # meters to millimeters
    ("mm", "m"): 1 / 1_000,       # millimeters to meters
    ("MPa", "Pa"): 1_000_000,     # megapascals to pascals
    ("Pa", "MPa"): 1 / 1_000_000, # pascals to megapascals
    ("mm2", "m2"): 1 / 1_000_000, # square millimeters to square meters
    ("m2", "mm2"): 1_000_000,     # square meters to square millimeters
}


def convert(value, from_unit, to_unit):
    """
    Convert between a predefined set of unit pairs.
    
    Args:
        value: numerical value to convert
        from_unit: string representing source unit
        to_unit: string representing target unit
    
    Returns:
        converted value
    
    Raises:
        ValueError if the conversion pair is not defined
    """
    key = (from_unit, to_unit)
    if key not in UNIT_FACTORS:
        raise ValueError(f"Conversion from {from_unit} to {to_unit} not defined")
    return value * UNIT_FACTORS[key]


# Example: Convert a list of lengths from meters to millimeters
example_lengths_m = [2.5, 3.0, 4.75]

# List comprehension applies convert() to each item in the list
converted_mm = [convert(length, "m", "mm") for length in example_lengths_m]

# zip() pairs up elements from two lists for easy iteration
for original, converted in zip(example_lengths_m, converted_mm):
    print(f"{original:.2f} m -> {converted:.0f} mm")

## 3. Working with DataFrames: Load Summary Table

**Key Python Concepts:**
- **Lists of dictionaries**: A common pattern for tabular data `[{key: value}, {key: value}, ...]`
- **Pandas DataFrames**: Table structures similar to Excel spreadsheets
- **Creating calculated columns**: Add new columns based on existing ones
- **DataFrame methods**: `.sum()` to total a column, `display()` to show formatted output

**Practical Application:**
In structural design, we often need to organize and calculate load combinations. This example shows how to:
1. Store load data in a structured format (list of dictionaries)
2. Convert it to a DataFrame for easy manipulation
3. Add calculated columns (factored loads)
4. Summarize totals

**Why use pandas?**
DataFrames make it easy to work with tabular data, filtering, sorting, calculating, and exporting to Excel or CSV. Much more powerful than managing lists manually.

**Note:** The load factors shown here (1.2D, 1.6L) are illustrative. Always use code-required combinations for real design.

In [ ]:
# Create a list of dictionaries to represent load cases
# Each dictionary is one row with keys as column names
load_cases = [
    {"name": "Dead", "magnitude_kN": 8.0, "factor": 1.2},
    {"name": "Live", "magnitude_kN": 5.5, "factor": 1.6},
    {"name": "Roof", "magnitude_kN": 2.1, "factor": 1.0},
]

# Convert the list to a pandas DataFrame
df_loads = pd.DataFrame(load_cases)

# Create a new calculated column by multiplying existing columns
# This is similar to adding a formula in Excel
df_loads["factored_kN"] = df_loads["magnitude_kN"] * df_loads["factor"]

# Display the table in a nicely formatted way
display(df_loads)

# Calculate column totals using the .sum() method
print("\n--- Load Summaries ---")
print(f"Total service load: {df_loads['magnitude_kN'].sum():.1f} kN")
print(f"Total factored load: {df_loads['factored_kN'].sum():.1f} kN")

## 4. Iteration and Formatting: Axial Stress Check

**Key Python Concepts:**
- **For loops**: Iterating through a list with `for item in list:`
- **Accessing dictionary values**: Using bracket notation `member["key"]` or dot notation (for some objects)
- **Percentage formatting**: Using `:.2%` in f-strings to display as percentage
- **Function composition**: Calling one function with results from another function

**Practical Application:**
This example demonstrates a common engineering workflow:
1. Define a helper function (`axial_stress`) that uses our `convert()` function
2. Create a list of trial members to check
3. Loop through each member, calculate stress, and check against allowable
4. Format output for easy review

**Why loop through data?**
Instead of repeating the same calculation manually for each member, we store member data in a list and process it systematically. This approach scales easily, add 100 members and the code remains the same.

**Note:** This example shows compression stress; always verify buckling, local buckling, and other limit states in real design.

In [ ]:
def axial_stress(force_kN, area_mm2):
    """
    Calculate axial stress given force and area.
    
    This function demonstrates composing multiple unit conversions:
    1. Convert force from kN to N
    2. Convert area from mm^2 to m^2
    3. Calculate stress = force / area in Pa
    
    Returns:
        stress in Pa (N/m^2)
    """
    force_N = convert(force_kN, "kN", "N")
    area_m2 = convert(area_mm2, "mm2", "m2")
    return force_N / area_m2  # Pa


# Define allowable stress
allowable_stress_MPa = 180

# Create a list of trial members (each is a dictionary)
trial_members = [
    {"label": "Column A", "force_kN": 650, "area_mm2": 32_000},
    {"label": "Column B", "force_kN": 540, "area_mm2": 25_000},
    {"label": "Column C", "force_kN": 420, "area_mm2": 20_000},
]

print("=" * 70)
print("AXIAL STRESS CHECK")
print(f"Allowable stress: {allowable_stress_MPa} MPa")
print("=" * 70)

# Loop through each member and perform the stress check
for member in trial_members:
    # Calculate stress in Pa
    stress_Pa = axial_stress(member["force_kN"], member["area_mm2"])
    
    # Convert to MPa for display
    stress_MPa = convert(stress_Pa, "Pa", "MPa")
    
    # Calculate utilization ratio (actual / allowable)
    ratio = stress_MPa / allowable_stress_MPa
    
    # Print formatted results
    # {:.1f} = one decimal place, {:.2%} = percentage with two decimals
    print(f"{member['label']:10} | {stress_MPa:6.1f} MPa | {ratio:6.2%} of allowable")

## 5. Visualization with Matplotlib: RC Beam Deflection Analysis

**Key Python Concepts:**
- **NumPy arrays**: `np.linspace()`, `np.minimum()`, `np.maximum()`, `np.argmin()` for numerical operations
- **Matplotlib plotting**: Creating figures, axes, and plots with `plt.subplots()`, `ax.plot()`, `ax.fill_between()`
- **Advanced calculations**: Implementing ACI 318 Branson's equation for effective moment of inertia
- **Quadratic formula**: Solving for neutral axis location in cracked transformed section
- **Array operations**: Vectorized calculations that work on entire arrays at once

**Practical Application:**
This comprehensive example demonstrates:
1. **Immediate deflection** of a reinforced concrete beam considering cracking (ACI 318)
2. **Branson's equation** for effective moment of inertia (Ie)
3. **Cracked section analysis** with transformed area method
4. **Parametric study** across a range of spans
5. **Visualization** comparing calculated deflection to serviceability limits (L/360)

**Why this matters:**
- Shows how to implement code equations in Python
- Demonstrates array-based calculations (more efficient than looping)
- Illustrates how visualization helps identify critical conditions
- Provides a template for similar parametric studies

**Learning Points:**
- Notice how variables are defined clearly at the top for easy modification
- Array operations (`span_m**2`) work element-wise, no loops needed
- The plot helps visualize where deflection exceeds the limit
- Comments explain the physics and code requirements

In [ ]:
"""
Reinforced Concrete Beam: Immediate Deflection with Cracking (ACI 318)

This script demonstrates:
- Implementation of ACI 318 deflection equations in Python
- Cracked section analysis using transformed area method
- Branson's equation for effective moment of inertia
- Parametric study and visualization with matplotlib
- Array-based calculations with NumPy (more efficient than loops)
"""

import numpy as np
import matplotlib.pyplot as plt

# ============================================================================
# SECTION 1: USER INPUTS
# ============================================================================
# All design parameters are defined here for easy modification

# Geometry (rectangular section)
width_mm = 200.0            # b: beam width (mm)
depth_mm = 400.0            # h: total depth (mm)
d_mm     = 360.0            # d: effective depth to centroid of tension steel (mm)

# Reinforcement
As_mm2   = 800.0            # As: area of tension reinforcement (mm^2)

# Materials
fc_MPa          = 30.0      # f'c: specified compressive strength of concrete (MPa)
E_concrete_GPa  = 30.0      # Ec: modulus of elasticity of concrete (GPa)
E_steel_GPa     = 200.0     # Es: modulus of elasticity of steel (GPa)

# Loading
uniform_load_kN_m = 12.0    # w: total unfactored service load (kN/m)

# Analysis parameters
span_m = np.linspace(3.0, 8.0, 30)  # Array of spans to analyze (m)
service_limit_ratio = 360.0         # Serviceability limit: L/360

# ============================================================================
# SECTION 2: UNIT CONVERSIONS
# ============================================================================
# Convert all inputs to consistent SI base units (m, Pa, N)

b   = width_mm / 1000.0      # m
h   = depth_mm  / 1000.0     # m
d   = d_mm      / 1000.0     # m
As  = As_mm2    / 1e6        # m^2

Ec  = E_concrete_GPa * 1e9   # Pa
Es  = E_steel_GPa   * 1e9    # Pa
n   = Es / Ec                # modular ratio (dimensionless)

w_N_m = uniform_load_kN_m * 1000.0  # N/m

# ============================================================================
# SECTION 3: GROSS SECTION PROPERTIES
# ============================================================================

Ig  = (b * h**3) / 12.0      # gross moment of inertia (m^4)
yt  = h / 2.0                # distance from centroid to extreme tension fiber (m)

# Modulus of rupture (ACI 318): fr = 0.62*sqrt(f'c) [MPa]
fr_MPa = 0.62 * np.sqrt(fc_MPa)
fr     = fr_MPa * 1e6        # convert to Pa

# Cracking moment: Mcr = fr * Ig / yt
Mcr = fr * Ig / yt           # N·m

# ============================================================================
# SECTION 4: CRACKED SECTION PROPERTIES
# ============================================================================
# Use transformed area method to find neutral axis and cracked inertia

# Solve for neutral axis depth 'x' from compression face:
# Equilibrium of first moments: (b/2)*x^2 = (n*As)*(d-x)
# Rearranging: (b/2)*x^2 + (n*As)*x - (n*As*d) = 0
# This is a quadratic equation: A*x^2 + B*x + C = 0

A_quad = b / 2.0
B_quad = n * As
C_quad = -n * As * d

# Quadratic formula: x = (-B + sqrt(B^2 - 4AC)) / (2A)
discriminant = B_quad**2 - 4.0 * A_quad * C_quad
if discriminant < 0:
    raise ValueError("Negative discriminant in NA solver; check inputs.")

x = (-B_quad + np.sqrt(discriminant)) / (2.0 * A_quad)  # m

# Ensure x is within physical bounds
x = np.clip(x, 1e-9, h - 1e-9)

# Cracked moment of inertia about the neutral axis:
# Icr = (b*x^3)/3 + n*As*(d-x)^2
Icr = (b * x**3) / 3.0 + n * As * (d - x)**2  # m^4

# ============================================================================
# SECTION 5: EFFECTIVE MOMENT OF INERTIA (BRANSON'S EQUATION)
# ============================================================================
# ACI 318: Ie = (Mcr/Ma)^3 * Ig + [1 - (Mcr/Ma)^3] * Icr  (for Ma >= Mcr)

# Service moment at midspan for simply supported beam with UDL
Ma = w_N_m * span_m**2 / 8.0      # N·m (this is an array)

# Calculate ratio Mcr/Ma, capped at 1.0 to avoid Ie > Ig
ratio = np.minimum(1.0, Mcr / np.maximum(Ma, 1e-12))  # avoid division by zero

# Branson's equation
Ie = ratio**3 * Ig + (1.0 - ratio**3) * Icr  # m^4 (array)

# Optional: Apply a lower bound on Ie (non-mandatory, engineering judgment)
# Uncomment the line below if desired:
# Ie = np.maximum(Ie, 0.35 * Ig)

# ============================================================================
# SECTION 6: DEFLECTION CALCULATION
# ============================================================================
# Immediate elastic deflection: δ_max = 5*w*L^4 / (384*E*Ie)

deflection_m  = (5.0 * w_N_m * span_m**4) / (384.0 * Ec * Ie)  # m
deflection_mm = deflection_m * 1000.0                           # mm

# Serviceability limit
limit_mm = (span_m * 1000.0) / service_limit_ratio  # mm

# Find critical span where deflection ≈ limit
critical_idx  = int(np.argmin(np.abs(deflection_mm - limit_mm)))
critical_span = span_m[critical_idx]

# ============================================================================
# SECTION 7: VISUALIZATION
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 5))

# Plot calculated deflection
ax.plot(span_m, deflection_mm, linewidth=2, label="Calculated deflection (ACI Ie)")

# Plot serviceability limit
ax.plot(span_m, limit_mm, linestyle="--", linewidth=2, 
        label=f"L/{int(service_limit_ratio)} limit")

# Shade acceptable zone (where deflection < limit)
ax.fill_between(span_m, 0.0, np.minimum(deflection_mm, limit_mm), 
                alpha=0.15, label="Acceptable zone")

# Formatting
ax.set_xlabel("Span (m)", fontsize=11)
ax.set_ylabel("Deflection (mm)", fontsize=11)
ax.set_title(
    f"RC Beam Immediate Deflection (ACI 318 Ie)\n"
    f"{width_mm:.0f}×{depth_mm:.0f} mm, As={As_mm2:.0f} mm² @ d={d_mm:.0f} mm, "
    f"w={uniform_load_kN_m:.1f} kN/m",
    fontsize=12, fontweight="bold"
)
ax.legend(loc="upper left", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# SECTION 8: TEXT OUTPUT SUMMARY
# ============================================================================

print("=" * 70)
print("RC BEAM DEFLECTION ANALYSIS (IMMEDIATE, ACI 318)")
print("=" * 70)
print(f"\nSection: b={width_mm:.0f} mm, h={depth_mm:.0f} mm, "
      f"d={d_mm:.0f} mm, As={As_mm2:.0f} mm²")
print(f"Materials: f'c={fc_MPa:.1f} MPa, Ec={E_concrete_GPa:.1f} GPa, "
      f"Es={E_steel_GPa:.1f} GPa")
print(f"Modular ratio: n = Es/Ec = {n:.2f}")
print(f"Load: w = {uniform_load_kN_m:.2f} kN/m")
print(f"\n--- Section Properties ---")
print(f"Gross inertia (Ig) = {Ig:.3e} m^4")
print(f"Cracking moment (Mcr) = {Mcr/1e3:.2f} kN·m")
print(f"Neutral axis depth (cracked, x) = {x*1000:.1f} mm")
print(f"Cracked inertia (Icr) = {Icr:.3e} m^4")
print(f"\n--- Critical Span ---")
print(f"Span where δ ≈ L/{int(service_limit_ratio)}: {critical_span:.2f} m")
print(f"Deflection at critical span: {deflection_mm[critical_idx]:.2f} mm")
print(f"Limit at critical span: {limit_mm[critical_idx]:.2f} mm")
print("=" * 70)